# Fine Tuning

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install unsloth langchain chromadb pdfplumber python-dotenv pandas transformers
!pip install -U langchain-community
!pip install sentence-transformers
!pip install bitsandbytes
!pip install accelerate
!pip install einops
!pip install numpy
# Import required libraries
import torch
import csv
import pandas as pd
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq, pipeline
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from trl import SFTTrainer
from langchain.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import uuid


In [ ]:
# 1. First load model and tokenizer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length=2048,
    load_in_4bit=torch.cuda.is_available(),
    # Add flash attention for optimization
    attn_implementation="flash_attention_2" if torch.cuda.is_available() else None,
)

# 2. Then apply chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

# 3. Prepare for inference
model = FastLanguageModel.for_inference(model)

# 4. Verify template application
test_messages = [{"role": "user", "content": "Hello"}]
try:
    tokenizer.apply_chat_template(test_messages)
except Exception as e:
    raise ValueError("Chat template application failed!") from e

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

# RAG

In [ ]:
# Define the method name
!pip install pdfplumber
!pip install chromadb
# method = "FineTuning + RAG"

# Load the PDF file
loader = PDFPlumberLoader("FAQ.pdf")
pages = loader.load()

# Split the text for better context continuity
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256, separators=["\n\n", "\n", ". ", "! ", "? ", " "])
chunks = text_splitter.split_documents(pages)

# embedding_function = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-small-en-v1.5",
#     model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
#     encode_kwargs={"normalize_embeddings": True},
# )
# Replace the existing embedding function with:
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 32  # Better for multilingual processing
    },
)

# Function to create vectorstore with normalized embeddings
def create_vectorstore(chunks, embedding_function, vectorstore_path):
    # Add explicit Chroma client configuration
    from chromadb.config import Settings
    client_settings = Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=vectorstore_path,
        anonymized_telemetry=False,
    )

    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        client_settings=client_settings,  # Add this line
        persist_directory=vectorstore_path,
        collection_metadata={
            "hnsw:space": "cosine",
            "dimension": 384
        },
    )
    vectorstore.persist()  # Explicitly persist the data
    return vectorstore

vectorstore_path = "/content/vectorstore"  # Use absolute path
import os
import shutil
import time

# Completely remove existing directory with proper permissions
if os.path.exists(vectorstore_path):
    shutil.rmtree(vectorstore_path, ignore_errors=True)
    time.sleep(2)  # Increase wait time


# Create fresh directory with write permissions
os.makedirs(vectorstore_path, exist_ok=True, mode=0o777)
# Set directory permissions explicitly
os.chmod(vectorstore_path, 0o777)
# Now create the vectorstore
try:
    vectorstore = create_vectorstore(
        chunks=chunks,
        embedding_function=embedding_function,
        vectorstore_path=vectorstore_path
    )
except Exception as e:
    print(f"Error creating vectorstore: {e}")
    # Fallback to in-memory
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        persist_directory=None
    )

# system_message = {
#     "role": "system",
#     "content": """You are an official advisor at IMT Mines Alès. Follow these rules:
# 1. Answer ONLY using the context below
# 2. If unsure, say "I don't have that information"
# 3. Keep responses under 3 sentences

# Context: {context}"""
# }

system_message = {
    "role": "system",
    "content": """You are an official advisor at IMT Mines Alès.
Answer STRICTLY using only this context: {context}
Rules:
1. If answer isn't in context, say "I don't have that information"
2. Keep responses under 3 sentences
3. Never mention you're an AI"""
}



PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    [system_message, {"role": "user", "content": "{question}"}],
    tokenize=False,
    add_generation_prompt=True,
)

# Add this validation after creating PROMPT_TEMPLATE
if "{context}" not in PROMPT_TEMPLATE or "{question}" not in PROMPT_TEMPLATE:
    raise ValueError("Prompt template missing required placeholders!")

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE,
)

# Set up retriever to get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.3,  # Lower for more deterministic answers
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,  # Important to avoid repeating input
)

llm = HuggingFacePipeline(pipeline=pipe)
# llm_chain = LLMChain(llm=llm, prompt=prompt)



<ipython-input-3-44809d93356b>:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Error creating vectorstore: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help!


<ipython-input-3-44809d93356b>:136: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
!pip install rank_bm25
# Create semantic retriever
semantic_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
texts = [doc.page_content for doc in chunks]

# Create keyword retriever
metadatas = [{"source": doc.metadata.get("source", "unknown")} for doc in chunks]
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 2

# Combine retrievers
ensemble_retriever = EnsembleRetriever(
    retrievers=[semantic_retriever, bm25_retriever],
    weights=[0.6, 0.4]
)

In [ ]:
# Correct validation function for NLI
def parse_response(full_response: str) -> str:
    # Extract only the assistant's response
    start_marker = "<|start_header_id|>assistant<|end_header_id|>"
    end_marker = "<|eot_id|>"

    start_idx = full_response.find(start_marker)
    end_idx = full_response.find(end_marker)

    if start_idx != -1 and end_idx != -1:
        return full_response[start_idx+len(start_marker):end_idx].strip()
    return full_response.split("assistant")[-1].strip()
def validate_response(response: str, context: str) -> str:
    # First check: Empty response
    if not response.strip():
        return "I don't have that information."

    # Second check: Direct contradiction
    validation_pipe = pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli",
        device=0 if torch.cuda.is_available() else -1,
    )

    result = validation_pipe(
        sequences=context,
        candidate_labels=["relevant", "irrelevant"],
        hypothesis_template="This context supports the response: '{}'",
        multi_label=False,
    )

    if result['labels'][0] == "irrelevant":
        return "I cannot confirm this information."

    return response
# Correct pipeline initialization
# validation_pipe = pipeline(
#     "zero-shot-classification",
#     model="facebook/bart-large-mnli",
#     device=0 if torch.cuda.is_available() else -1,
#     hypothesis_template="This example is {}",
# )

In [ ]:
from transformers import StoppingCriteria

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_token_ids):
        self.stop_token_ids = stop_token_ids

    def __call__(self, input_ids, scores, **kwargs):
        return any(token in self.stop_token_ids for token in input_ids[0][-3:])

# Add this right after the class definition:
stop_token_ids = [tokenizer.eos_token_id] + [
    tokenizer.encode(token, add_special_tokens=False)[-1]  # Get last token ID
    for token in ["\nReferences:", "\nSource:"]
]
# Modified in Block 6 (Generation Pipeline)
def full_rag_pipeline(question):
    try:
        # 1. Retrieve context
        docs = ensemble_retriever.invoke(question)
        context = "\n".join([d.page_content for d in docs[:3]])  # Top 3 docs

        # 2. Format prompt
        messages = [
            {"role": "system", "content": f"Context: {context}"},
            {"role": "user", "content": question}
        ]
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
        )

        # 3. Tokenize inputs
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")

        # 4. Generate response
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=256,
            temperature=0.5,
            top_p=0.9,
            stopping_criteria=[StopOnTokens(stop_token_ids)],
            pad_token_id=tokenizer.eos_token_id,
        )

        # 5. Decode and validate
        response = tokenizer.decode(outputs[0], skip_special_tokens=False)
        clean_response = response.split("<|start_header_id|>assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip()

        return validate_response(clean_response, context)

    except Exception as e:
        print(f"Error: {str(e)}")
        return "I'm unable to answer that right now."

In [ ]:
response = full_rag_pipeline("IMT Mines Ales est situe ou?")
print(response)

IMT Mines Alès est situé en Alès, dans le sud de la France.


In [ ]:
# Add these imports at the top
from typing import Dict, List
from langchain.schema import HumanMessage, AIMessage

# 1. Conversation State Manager
class ChatSession:
    def __init__(self):
        self.history: List[Dict] = []
        self.context_window = 2048  # Match model's max_seq_length

    def add_interaction(self, question: str, answer: str):
        self.history.extend([
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ])

    def get_recent_history(self, token_limit: int = 512) -> str:
        """Return truncated conversation history"""
        history_text = "\n".join(
            f"{msg['role'].capitalize()}: {msg['content']}"
            for msg in self.history[-4:]  # Last 2 exchanges
        )
        return self._truncate_text(history_text, token_limit)

    def _truncate_text(self, text: str, max_tokens: int) -> str:
        tokens = tokenizer.encode(text, add_special_tokens=False)
        return tokenizer.decode(tokens[-max_tokens:], skip_special_tokens=True)

# 2. Enhanced RAG Pipeline with History
def interactive_rag_pipeline(question: str, session: ChatSession) -> str:
    # Incorporate conversation history in retrieval
    combined_query = f"{session.get_recent_history()}\n{question}"

    # Retrieve documents with history context
    docs = ensemble_retriever.invoke(combined_query)
    context = "\n".join([d.page_content for d in docs[:3]])

    # Build messages with history
    messages = [
        {
            "role": "system",
            "content": f"""Current conversation:
{session.get_recent_history()}

Context:
{context}"""
        },
        {"role": "user", "content": question}
    ]

    # Generate response
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to("cuda")

    outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=256,
        temperature=0.5,
        stopping_criteria=[StopOnTokens(stop_token_ids)],
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    clean_response = response.split("assistant<|end_header_id|>")[-1].split("<|eot_id|>")[0].strip()

    # Validate and format
    final_response = validate_response(clean_response, context)
    session.add_interaction(question, final_response)

    return final_response

# 3. Interactive Chat Interface
def chat_interface():
    session = ChatSession()
    print("Chatbot: Welcome to IMT Mines Alès assistant! Type '/exit' to end.")

    while True:
        try:
            user_input = input("\nYou: ")
            if user_input.lower() == "/exit":
                break

            if user_input.lower() == "/reset":
                session = ChatSession()
                print("Chatbot: Conversation history reset!")
                continue

            response = interactive_rag_pipeline(user_input, session)
            print(f"\nChatbot: {response}")

        except KeyboardInterrupt:
            break
        except Exception as e:
            print(f"\nChatbot: Sorry, I encountered an error. Please try again.")

    print("\nChatbot: Thank you for using our service!")

# Run the chat interface
chat_interface()

Chatbot: Welcome to IMT Mines Alès assistant! Type '/exit' to end.

You: bonjour


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


Chatbot: Bonjour! Comment puis-je vous aider aujourd'hui?

You: IMT Mines Ales est situe ou?

Chatbot: IMT Mines Alès est situé dans la ville d'Alès, dans le département du Gard, en région Occitanie, dans le sud de la France.

You: C'est qui Setthika?

Chatbot: Je suis désolé, je n'ai pas trouvé d'information sur une personne nommée Setthika dans le texte fourni. Puis-je vous aider avec autre chose?

You: Tu connais Dr Somreth?

Chatbot: Je n'ai pas trouvé d'information sur une personne nommée Dr Somreth dans le texte fourni. Puis-je vous aider avec autre chose?

You:  DO I NEED A FRENCH SOCIAL INSURANCE?

Chatbot: Oui, vous avez besoin d'une assurance sociale française. Vous serez instruit à la fin de votre étude (au cours de la première semaine avec l'aide de tout le personnel administratif).

You: En anglais svp

Chatbot: Here's the same text in English:

Do I need a French social insurance?
Yes, you have to get a French social insurance. You will be instructed about it at the begi

# Gradio